In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# Load Data and Correct Errors

In [50]:
coup = pd.read_csv("data/Coup_Data_v2.0.0.csv")
# WHO MISPELLS ECUADOR!
coup['country'] = np.where(coup.country == 'Ecudaor', 'Ecuador', coup.country)
coup['country'] = np.where(coup.country == 'Surinam', 'Suriname', coup.country)

# Merge Regional Data

In [51]:
# Get desired columns
cols = ['region', 'sub-region', 'name']

# Deal with different naming conventions
countries = pd.read_csv('data/country_info.csv', usecols=cols)
countries.rename(columns={'name': 'country'},inplace=True)
countries['country'] = np.where(countries.country == 'Timor-Leste', 'East Timor',
                               np.where(countries.country == 'Venezuela (Bolivarian Republic of)', 'Venezuela',
                                       np.where(countries.country== "Lao People's Democratic Republic", 'Laos',
                                               np.where(countries.country == "Viet Nam", 'Republic of Vietnam',
                                                       np.where(countries.country == 'Bolivia (Plurinational State of)', 'Bolivia',
                                                               np.where(countries.country == 'Congo, Democratic Republic of the', 'Democratic Republic of the Congo',
                                                                        np.where(countries.country == "Korea, Republic of", 'Republic of Korea',
                                                                                 np.where(countries.country =='Slovakia','Czechoslovakia', 
                                                                                          np.where(countries.country == 'Germany', 'German Democratic Republic',
                                                                                                   np.where(countries.country == 'Iran (Islamic Republic of)', 'Iran', 
                                                                                                            np.where(countries.country == 'Romania', 'Rumania', 
                                                                                                                     np.where(countries.country =='Russian Federation', 'Russia', 
                                                                                                                              np.where(countries.country == 'Syrian Arab Republic', 'Syria', 
                                                                                                                                       np.where(countries.country == 'Eswatini', 'Swaziland', 
                                                                                                                                                np.where(countries.country == 'Tanzania, United Republic of', 'Tanzania', 
                                                                                                                                                np.where(countries.country == "Côte d'Ivoire", "Cote d'Ivoire", countries.country))))))))))))))))


coup = pd.merge(coup,countries, how='left', on=['country'])

coup['sub-region'] = np.where(coup.country == 'Ivory Coast', 'Sub-Saharan Africa', 
                                np.where(coup.country == 'Kyrgyz Republic', 'Central Asia', 
                                         np.where(coup.country == 'USSR', 'Eastern Europe', 
                                                  np.where(coup.country == 'Yemen Arab Republic','Western Asia', 
                                                           np.where(coup.country == 'Yemen PDR', 'Western Asia',coup['sub-region'])))))

coup['region'] = np.where(coup.country == 'Ivory Coast', 'Africa', 
                                np.where(coup.country == 'Kyrgyz Republic', 'Asia', 
                                         np.where(coup.country == 'USSR', 'Europe', 
                                                  np.where(coup.country == 'Yemen Arab Republic','Asia', 
                                                           np.where(coup.country == 'Yemen PDR', 'Asia',coup.region)))))

# Merge Gini Data

In [52]:
gini = pd.read_csv('data/gini_2100.csv')
gini['country'] = np.where(gini.country == 'Congo, Rep.', 'Congo',
                           np.where(gini.country == 'Congo, Dem. Rep.', 'Democratic Republic of the Congo',
                                    np.where(gini.country == 'Timor-Leste', 'East Timor', 
                                             np.where(gini.country=='South Korea', 'Republic of Korea',
                                                      np.where(gini.country=='Lao','Laos', 
                                                               np.where(gini.country == 'United States', 'United States of America', 
                                                                        np.where(gini.country == 'Romania', 'Rumania', 
                                                                                 np.where(gini.country == 'Vietnam', 'Republic of Vietnam', 
                                                                                          np.where(gini.country == 'Czech Republic', 'Czechoslovakia', 
                                                                                                   np.where(gini.country == 'Eswatini', 'Swaziland', 
                                                                                                            np.where(gini.country == 'Germany', 'German Democratic Republic', gini.country)))))))))))

gini_sub = gini[(gini.country == 'Russia') | (gini.country == 'Yemen') | (gini.country == 'Kyrgyz Republic') | (gini.country == "Cote d'Ivoire")].copy()
gini_sub['country']  = np.where(gini_sub.country == 'Russia', 'USSR', 
                               np.where(gini_sub.country == "Cote d'Ivoire", 'Ivory Coast', 
                                       np.where(gini_sub.country =='Kyrgyz Republic', 'Kyrgyzstan', 
                                               np.where(gini_sub.country == 'Yemen', 'Yemen Arab Republic', gini_sub.country))))
gini_sub2 = gini[gini.country == 'Yemen'].copy()
gini = pd.concat([gini,gini_sub])

gini_sub2['country'] = np.where(gini_sub2.country =='Yemen', 'Yemen PDR', gini_sub2.country)
gini = pd.concat([gini, gini_sub2])
gini_countries = gini.country.to_list()
coup_countries = coup.country.to_list()

years = gini.columns.to_list()[1:]
coup_countries = coup.country.unique().tolist()
country_dfs = []
for c in coup_countries:
    ginis = gini[(gini.country == c)].values.flatten().tolist()[1:]
    for y in years:
        data = {'country': [c],
                'year': [y],
                'gini': ginis[years.index(y)]}
        df = pd.DataFrame(data)
        country_dfs.append(df)
gini2 = pd.concat(country_dfs, ignore_index=True)
gini2['year'] = gini2.year.astype('int')

coup = pd.merge(coup, gini2, how='left', on=['country', 'year'])

# Merge Life Expectancy Data

In [53]:
life = pd.read_csv("data/life_expectancy_years.csv")

# Countries under different names
problems = {"East Timor":"Timor-Leste",
            "Kyrgyzstan":"Kyrgyz Republic",
            "Laos":"Lao",
            "Ivory Coast":"Cote d'Ivoire",
            "Democratic Republic of the Congo":"Congo, Dem. Rep.",
            "Congo":"Congo, Rep.",
            "Czechoslovakia":"Czech Republic",
            "German Democratic Republic":"Germany",
            "Republic of Korea":"South Korea",
            "Republic of Vietnam":"Vietnam",
            "Rumania":"Romania",
            "Swaziland":"Eswatini",
            "United States of America":"United States",
            "USSR":"Russia",
            "Yemen Arab Republic":"Yemen",
            "Yemen PDR":"Yemen"
}

exp_list = []
prob = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in life["country"].tolist():
        idx = life.index[life['country']== curr_country].tolist()[0]
        exp = life[str(curr_year)][idx]
        exp_list.append(exp)
        
    else:
        exp_list.append(np.nan)  

coup["life_expectancy"] = exp_list

# Merge Mean Household Income Data

In [54]:
hhinc = pd.read_csv("data/mhhinc.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in hhinc["country"].tolist():
        idx = hhinc.index[hhinc['country']== curr_country].tolist()[0]
        exp = hhinc[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan) 
        
for i in range(len(exp_list)):
    if type(exp_list[i])==str and "k" in exp_list[i]:
        curr_exp = float(exp_list[i][:-1])*1000
        exp_list[i] = curr_exp
coup["hhinc"] = exp_list

# Merge Total Population

In [55]:
pop = pd.read_csv("data/population_total.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in pop["country"].tolist():
        idx = pop.index[pop['country']== curr_country].tolist()[0]
        exp = pop[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan)   
        
for i in range(len(exp_list)):
    if type(exp_list[i])==str and "k" in exp_list[i]:
        curr_exp = float(exp_list[i][:-1])*1000
        exp_list[i] = curr_exp
    if type(exp_list[i])==str and "M" in exp_list[i]:
        curr_exp = float(exp_list[i][:-1])*1000000
        exp_list[i] = curr_exp
    if type(exp_list[i])==str and "B" in exp_list[i]:
        curr_exp = float(exp_list[i][:-1])*1000000000
        exp_list[i] = curr_exp
    
coup["population"] = exp_list

# Merge Population Growth Data

In [56]:
growth = pd.read_csv("data/population_growth_annual_percent.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_year < 1960:
        curr_year = 1960
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in growth["country"].tolist():
        idx = growth.index[growth['country']== curr_country].tolist()[0]
        exp = growth[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan)    

coup["pop_growth"] = exp_list

# Merge Sex Ratio Data

In [57]:
sex = pd.read_csv("data/sex_ratio_all_age_groups.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_year < 1949:
        curr_year = 1949
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in sex["country"].tolist():
        idx = sex.index[sex['country']== curr_country].tolist()[0]
        exp = sex[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan)    

for i in range(len(exp_list)):
    exp_list[i] = int(exp_list[i])/100
    
coup["sex_ratio"] = exp_list

# Merge GDP Per Capita

In [58]:
gdppc = pd.read_csv("data/income_per_person_gdppercapita_ppp_inflation_adjusted.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in gdppc["country"].tolist():
        idx = gdppc.index[gdppc['country']== curr_country].tolist()[0]
        exp = gdppc[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan)    
        
for i in range(len(exp_list)):
    if type(exp_list[i])==str and "k" in exp_list[i]:
        curr_exp = float(exp_list[i][:-1])*1000
        exp_list[i] = curr_exp
coup["gdppc"] = exp_list

# Merge GDP Growth

In [59]:
gdpgrwth = pd.read_csv("data/gdp_total_yearly_growth.csv")

exp_list = []
for i in range(np.shape(coup)[0]):
    curr_country = coup["country"][i]
    curr_year = coup["year"][i]
    if curr_year >2012:
        curr_year = 2012
    if curr_country in problems.keys():
        curr_country = problems[curr_country]
    if curr_country in gdpgrwth["country"].tolist():
        idx = gdpgrwth.index[gdpgrwth['country']== curr_country].tolist()[0]
        exp = gdpgrwth[str(curr_year)][idx]
        exp_list.append(exp)
    else:
        exp_list.append(np.nan)    
        
coup["gdpgrwth"] = exp_list


# Merge Previous Coup Data

In [60]:
country_name_change = {"Kyrgyzstan":"Kyrgyz Republic",
            "Czechoslovakia":"Czech Republic",
            "German Democratic Republic":"Germany",
            "USSR":"Russia",
            "Kyrgyz Republic":"Kyrgyzstan",
            "Czech Republic":"Czechoslovakia",
            "Germany":"German Democratic Republic",
            "Russia":"USSR"
}

id_1 = []
id_2 = []
id_3 = []
id_4 = []
id_5 = []
id_6 = []
id_7 = []

for i in range(np.shape(coup)[0]):
    curr_year = coup["year"][i]
    curr_country = coup["country"][i]
    country_list = [curr_country]
    
    if curr_country in country_name_change.keys():
        country_list.append(country_name_change[curr_country])
        id_6.append(1)
    
    elif "Yemen" in curr_country:
        country_list = ["Yemen", "Yemen Arab Republic", "Yemen PDR"]
        id_6.append(2)
        
    else: 
        id_6.append(0)
        
    # taking country subset
    curr_subset = coup[coup["country"].isin(country_list)]
    curr_subset = curr_subset[curr_subset["year"]<curr_year]
    
    # previous coup attempts (all time)
    id_1.append(np.shape(curr_subset)[0])
    
    # previous coup attempts (5 yrs)
    subsubset = curr_subset[curr_subset["year"]>curr_year-5]
    id_2.append(np.shape(subsubset)[0])
    
    # previous coup failures
    id_3.append(np.sum(curr_subset["unrealized"]))
    
    # previous coup successes
    id_4.append(np.sum(curr_subset["realized"]))
    
    # previous leader harm
    id_5.append(np.sum(curr_subset["injured"])+np.sum(curr_subset["killed"]))
    
    # previous leader court
    id_7.append(np.sum(curr_subset["harrest"])+np.sum(curr_subset["jailed"])+np.sum(curr_subset["tried"]))

# Merge and Impute Polity 5 Data

In [ ]:
'''
-Cases of -77 indicate interregum periods (period between ruling system of power), so we indicate 0 to denote that the scores are not applicable in ,
-Cases of -88 indicate transition periods between standing regimes, so we resolve this by imputing the average score of the nation in each column as a rough estimator
of political conditions during the transition period
-Cases of -66 means that a governing system is missing (so we impute scores of 0 to denote not applicable) (Page 17 of Polity5 Guide)'''

polity = pd.read_csv('data/p5v2018.csv')

polity = polity.rename(columns={
    'ccode':'cowcode'
})

polity = polity[['year', 'cowcode', 'democ', 'autoc', 'polity2', 'durable', 'xrcomp', 'xropen', 'xconst', 'parreg', 'parcomp', 'exrec']]

coup = coup.merge(polity, on=['cowcode', 'year'], how='left', suffixes=(False, False))

country_ids = ['coup_id', 'cowcode', 'country', 'year']
polity_cols = ['democ', 'autoc', 'polity', 'polity2', 'durable', 'xrreg', 'xrcomp',
              'xropen', 'xconst', 'parreg', 'parcomp', 'exrec']


#Impute 0's for -77 code

coup["standing_regime"] = np.where(coup["democracy"] == -77, 0, 1)
for feature in polity_cols:
    coup[feature].mask(coup[feature] == -77, 0, inplace=True)
display(coup)


# Impute 0s for -66 code

coup["standing_regime"].mask(coup['democ'] == -66, 0, inplace=True)
for feature in polity_cols:
    coup[feature].mask(coup[feature] == -66, 0, inplace=True)
    
coup = coup.rename(columns={'democ': 'democracy', 'autoc': 'autocracy',
                        'polity2':'polity2_score', 'durable':'years_of_regime_pow', 'xrcomp':'exec_competitive_score', 
                        'xropen': 'exec_openness_score', 'xconst': 'exec_constraint_score', 'parreg' : 'political_regularity', 
                        'parcomp' : 'political_competitiveness','exrec': 'exec_recruitment'})

#Impute data for -88 code

p5_df = pd.read_excel("data/p5v2018.xls")

relevant_p5 = p5_df[["country"] + polity_cols]

coup["standing_regime"].mask(coup['democracy'] == -88, 0, inplace=True)

country_list = [x for x in coup['country'].unique() if x in relevant_p5['country'].unique()]

for country in country_list:
    country_df = relevant_p5[relevant_p5['country'] == country]

    for i in range(len(polity_cols)):
        feature_avg = int(np.round(np.mean(country_df[country_df[polity_cols[i]] != -88][polity_cols[i]])))
        
        # Impute means of non-standing government and missing observation
        coup[polity_cols[i]].mask(coup[polity_cols[i]] == -88, feature_avg, inplace=True)

        coup[polity_cols[i]].mask(coup[polity_cols[i]].isna(), feature_avg, inplace=True)


coup = coup.rename(columns={'democ': 'democracy', 'autoc': 'autocracy',
                        'polity2':'polity2_score', 'durable':'years_of_regime_pow', 'xrcomp':'exec_competitive_score', 
                        'xropen': 'exec_openness_score', 'xconst': 'exec_constraint_score', 'parreg' : 'political_regularity', 
                        'parcomp' : 'political_competitiveness','exrec': 'exec_recruitment'})


# Merge and Impute Political Violence Data

In [ ]:

poly_vio_df = pd.read_csv('MEPVv2018.xls')

#Grenada, the Maldives, Seychelles, Sao Tome and Principe, and Dominica are not present, Egypt(1947) is not present

alt_code_dict = {769:770, 365:393, 364:365, 529:530, 626:625, 525:626}

poly_vio_df['ccode'] = poly_vio_df['ccode'].replace(alt_code_dict)

display(poly_vio_df[poly_vio_df['ccode'] == 770])

poly_vio_df['cowcode'] = poly_vio_df.ccode

relevant_polyv_cols = ['cowcode', 'year', 'intind',
       'intviol', 'intwar', 'civviol', 'civwar', 'ethviol', 'ethwar', 'inttot',
       'civtot', 'actotal', 'nborder']
new_coup = pd.merge(coup, poly_vio_df[relevant_polyv_cols], how='left', on=['cowcode', 'year'])

missing = new_coup[new_coup.civtot.isna() == True]

fill_list = []

for i in range(len(missing)):
       curr_code = new_coup[new_coup.civtot.isna() == True].iloc[i]['cowcode']
       curr_year = new_coup[new_coup.civtot.isna() == True].iloc[i]['year']
       
       new_entry = poly_vio_df[(poly_vio_df['cowcode'] == curr_code) & (poly_vio_df['year'] == curr_year-1)][relevant_polyv_cols]
       
       if new_entry.empty:
              new_entry = poly_vio_df[(poly_vio_df['cowcode'] == curr_code) & (poly_vio_df['year'] == curr_year+1)][relevant_polyv_cols]
       if new_entry.empty:
              continue
       fill_list.append(new_entry.values.tolist())

new_coup.rename(columns={'intind':'indep_viol_mag', 'intviol':'intl_viol_mag',
       'intwar':'intl_war_mag', 'civviol':'civil_viol_mag', 'civwar':'civil_war_mag', 'ethviol':'ethnic_viol_mag',
       'ethwar':'ethnic_war_mag', 'inttot':'total_intl_viol_mag', 'civtot':'total_civil_viol_mag',
       'actotal': 'total_viol_mag' , 'nborder':'number_of_neighbors'})


code_list = [x[0][0] for x in fill_list]
year_list = [x[0][1] for x in fill_list]


change = new_coup['civviol'].isna()
for i in range(len(fill_list)):
    new_coup.loc[change & (new_coup['cowcode'] == code_list[i]) & (new_coup['year'] == year_list[i]+1), relevant_polyv_cols[2:]] = fill_list[i][0][2:]
    new_coup.loc[change & (new_coup['cowcode'] == code_list[i]) & (new_coup['year'] == year_list[i]-1), relevant_polyv_cols[2:]] = fill_list[i][0][2:]
 
# Switched cow code so DRC must be entered manually
new_coup.loc[change & (new_coup['cowcode'] == 480) & (new_coup['year'] == 2013), relevant_polyv_cols[2:]] = [0,0,0,0,5,1,0,0,6,6,8]

#The other 16 columns cannot be resolved and so it is we could consider dropping them

y2 = new_coup[new_coup.civtot.isna() == True]
display(y2)

new_coup = new_coup.drop(index=[15,309,335,402,403,404,405,572,573,574,742,743,745,746,747,748])

new_coup[new_coup['pop_growth'].isna() == True]['pop_growth'] = 1.35

y3 = coup['pop_growth'].isna().sum()
display(coup.isna().sum(axis=0))
y3[y3 > 0]

new_coup.to_csv("coup_no_missingness.csv")

In [24]:
# Create dummy indicators for columns with string values
def dummy_gen(column):
    unique_vals = coup[column].unique().tolist()
    for u in unique_vals:
        coup[u] = np.where(coup[column] == u, 1, 0)
    coup.drop(columns=[column], inplace=True)